In [11]:
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import display, clear_output

class ED:

    def __init__(self, num_individuos, num_dimensiones, paso, tasa_cruce, problema, num_generaciones):
        self.num_individuos = num_individuos
        self.num_dimensiones = num_dimensiones
        self.paso = paso
        self.tasa_cruce = tasa_cruce
        self.problema = problema
        self.num_generaciones = num_generaciones
        self.rango = self.problema.MAX_VALUE - self.problema.MIN_VALUE

    def ejecutar(self):
        self.individuos = np.random.random(size=(self.num_individuos, self.num_dimensiones)) * self.rango + self.problema.MIN_VALUE
        fitness = np.apply_along_axis(self.problema.fitness, 1, self.individuos)

        mejor = np.argmin(fitness)
        mejor_fitness = fitness[mejor]

        generaciones = []

        for generacion in range(self.num_generaciones):
            u = np.empty_like(self.individuos)
            v = np.empty_like(self.individuos)

            for i in range(self.num_individuos):
                r1, r2, r3 = np.random.choice(np.delete(np.arange(self.num_individuos), i), 3, replace=False)

                v[i] = self.individuos[r1] + self.paso * (self.individuos[r2] - self.individuos[r3])

                tr = np.random.randint(0, self.num_dimensiones)
                rcj = np.random.random(self.num_dimensiones)

                u[i] = np.where(rcj < self.tasa_cruce, v[i], self.individuos[i])
                u[i][tr] = v[i][tr]

            fitness_u = np.apply_along_axis(self.problema.fitness, 1, u)
            mejor_u = np.argmin(fitness_u)

            if fitness_u[mejor_u] < mejor_fitness:
                mejor = mejor_u
                mejor_fitness = fitness_u[mejor]

            self.individuos = np.where(fitness_u[:, np.newaxis] < fitness[:, np.newaxis], u, self.individuos)

            if generacion % 100 == 0:
                print('Generación ', generacion, ':', mejor_fitness)
                generaciones.append(mejor_fitness)

        return generaciones

In [12]:
#Funciones
class Sphere:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma):
      z = 0
      for alelo in cromosoma:
          z += alelo**2
      return z

class Rosenbrock:
  MIN_VALUE = -2.048
  MAX_VALUE = 2.048
  def __init__(self):
    pass
  def fitness(self, vector):
    z = 0
    for dimension in range(len(vector)-1):
      z += 100 * (vector[dimension + 1] - vector[dimension]**2)**2 + (vector[dimension]-1)**2
    return z

class Rastrigin:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma, C = 10):
    z = C * len(cromosoma) #10n
    for alelo in cromosoma:
      z += alelo**2 - C * np.cos(2 * np.pi * alelo)
    return z
class Quartic:
  MIN_VALUE = -1.28
  MAX_VALUE = 1.28
  def __init__(self):
      pass
  def fitness(self, cromosoma):
    z = 0
    for alelo in cromosoma:
      z += alelo**4
    return z

In [13]:
#Función para graficar los promedios individualmente
def graficarIndividual(arreglo, nombre, dimensiones):
  datos_y = arreglo
  datos_x = range(len(datos_y))

  # Graficar la línea
  plt.plot(datos_x, datos_y, marker='o')

  plt.xticks(range(0, 20))

  plt.title(f'{nombre}: {dimensiones} dimensiones')

  # Agregar etiquetas a los ejes
  plt.xlabel('Eje X')
  plt.ylabel('Eje Y')

  # Mostrar la gráfica
  plt.show()

In [14]:
#Función para graficae los promedios en conjunto
def graficarConjunto(arregloPromedios):
    colores = ['red', 'blue', 'green']
    dimensiones = [2, 4, 8]
    datos_x = range(len(arregloPromedios[0]))  # asumimos que todos los arreglos tienen la misma longitud

    # Verificamos si algún valor en los arreglos internos es mayor que 100
    valorMaximo = any(max(arreglo) > 100 for arreglo in arregloPromedios)

    if valorMaximo:
      plt.ylim(0, 100)  # Establece el límite superior en el eje Y a 100
    #---------------------------------------------------------------------

    # Graficar cada arreglo en la misma gráfica con colores diferentes
    for i, arreglo in enumerate(arregloPromedios):
        color = colores[i] if colores else None
        plt.plot(datos_x, arreglo, marker='o', label=f'{dimensiones[i]} dimensiones', color=color)

    plt.xticks(range(0, 20))

    plt.title('Promedios de todas las dimensiones')

    # Agregar etiquetas a los ejes
    plt.xlabel('Eje X')
    plt.ylabel('Eje Y')

    # Mostrar la gráfica
    plt.legend()
    plt.show()

In [15]:
def main():
  # Funciones de optimización disponibles
  optimization_functions = {
    'Sphere': Sphere(),
    'Rosenbrock': Rosenbrock(),
    'Rastrigin': Rastrigin(),
    'Quartic': Quartic()
  }

  # Arreglo para graficar los promedios en conjunto
  conjuntoPromedios = []
  # Dimensiones
  availableDimensions = [2, 4, 8]

  # Widget de selección de función
  function_selector = widgets.Dropdown(
    options=list(optimization_functions.keys()),
    value='Sphere',
    description='Función:'
  )

  # Función que se llama al hacer clic en el botón
  def on_button_click(b):
    output.clear_output()
    selected_function_name = function_selector.value
    selected_function_class = optimization_functions[selected_function_name]
    selected_function_instance = optimization_functions[selected_function_name]
    rango = selected_function_instance.MAX_VALUE - selected_function_instance.MIN_VALUE
    cantidad_individuos = 30
    paso=0.5
    tasa_cruce=0.7
    generaciones = 2000

    with output:
        # Ejecutar la función para cada dimensión disponible
        for dimensions in availableDimensions:
          mejores_historicosG = []
          print(f"\nFunción seleccionada: {function_selector.value}")
          print(f"Dimensiones: {dimensions}")
          for i in range(5):
              print(f"\nRun número {i+1}")
              ed = ED(cantidad_individuos, dimensions, paso, tasa_cruce, selected_function_instance, generaciones)
              mejores_historicosG.append(ed.ejecutar())
          #Promediar
          mejores_historicosG = [sum(x) / len(x) for x in zip(*mejores_historicosG)]
          conjuntoPromedios.append(mejores_historicosG)
          print(f'\nPromedios de los mejores historicos: \n{mejores_historicosG}\n')
          #Graficar individualmente
          graficarIndividual(mejores_historicosG, function_selector.value, dimensions)
        # Graficar promedios en conjunto
        print('\nGráica con todos los promedios:\n')
        graficarConjunto(conjuntoPromedios)

  # Widget de botón
  button = widgets.Button(description="Ejecutar")
  output = widgets.Output()

  # Asignar la función al evento de clic del botón
  button.on_click(on_button_click)

  # Mostrar widgets
  display(function_selector, button, output)

In [ ]:
main()